In [1]:
                            
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

'''import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
'''

"import os\nfor dirname, _, filenames in os.walk('/kaggle/input'):\n    for filename in filenames:\n        print(os.path.join(dirname, filename))\n"

In [2]:
!pip install transformers datasets tokenizers seqeval -q

In [3]:
pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.5 MB/s eta 0:00:00eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


* O means the word doesn’t correspond to any entity.
* B-PER/I-PER means the word corresponds to the beginning of/is inside a person entity.
* B-ORG/I-ORG means the word corresponds to the beginning of/is inside an organization entity.
* B-LOC/I-LOC means the word corresponds to the beginning of/is inside a location entity.
* B-MISC/I-MISC means the word corresponds to the beginning of/is inside a miscellaneous entity.

In [4]:
import datasets 
import numpy as np 
from transformers import BertTokenizerFast 
from transformers import DataCollatorForTokenClassification 
from transformers import AutoModelForTokenClassification 
import json
import pandas as pd

In [6]:
model_path = "CAMeL-Lab/bert-base-arabic-camelbert-mix"

In [7]:
tokenizer = BertTokenizerFast.from_pretrained(model_path) 

In [8]:
from datasets import load_dataset

In [9]:
def process_json_file(file_path, tag_to_int=None, label_ids=None):
    # Load the data from the JSON file
    with open(file_path, 'r') as file:
        data = json.load(file)

        token_data = []
        label_data = []

        if tag_to_int is None:
            tag_to_int = {}
            label_ids = {}
            unique_id = 0
        else:
            unique_id = len(tag_to_int)

        for i in range(len(data)):
            token_list = []
            label_list = []
            for j in range(len(data[i]['tokens'])):
                token = data[i]['tokens'][j]['token']
                tag = data[i]['tokens'][j]['tags'][0]['value']

                # If tag is not in the dictionary, add it and assign a new unique integer
                if tag not in tag_to_int:
                    tag_to_int[tag] = unique_id
                    label_ids[unique_id] = tag
                    unique_id += 1

                # Use the integer value from the dictionary
                token_list.append(token)
                label_list.append(tag_to_int[tag])

            token_data.append(token_list)
            label_data.append(label_list)

        result_dict = {'tokens': token_data, 'labels': label_data}

        return result_dict, tag_to_int, label_ids


In [10]:
# Train dataset path
inputfile = '/kaggle/input/arabic-wojood-ner-2024/split70.json'
# Test Dataset Path
inputfile2= '/kaggle/input/arabic-wojood-ner-2024/split10.json'

In [13]:
# creating dataset from Json file and extracting tags and their IDs
dataset, tag_to_int, label_ids = process_json_file(inputfile)

In [12]:
test_dataset, _, _ = process_json_file(inputfile2, tag_to_int, label_ids)

In [18]:
dataset.keys()

dict_keys(['tokens', 'labels'])

In [19]:
label_ids

{0: 'O',
 1: 'B-CARDINAL',
 2: 'B-ORG',
 3: 'I-ORG',
 4: 'B-DATE',
 5: 'B-LANGUAGE',
 6: 'B-NORP',
 7: 'B-PERS',
 8: 'I-PERS',
 9: 'B-OCC',
 10: 'I-DATE',
 11: 'B-GPE',
 12: 'B-EVENT',
 13: 'I-EVENT',
 14: 'I-OCC',
 15: 'I-CARDINAL',
 16: 'B-FAC',
 17: 'I-FAC',
 18: 'B-LOC',
 19: 'B-MONEY',
 20: 'I-MONEY',
 21: 'B-ORDINAL',
 22: 'I-LANGUAGE',
 23: 'B-TIME',
 24: 'I-TIME',
 25: 'I-GPE',
 26: 'I-LOC',
 27: 'I-NORP',
 28: 'B-QUANTITY',
 29: 'B-PERCENT',
 30: 'I-PERCENT',
 31: 'B-UNIT',
 32: 'I-QUANTITY',
 33: 'B-CURR',
 34: 'B-WEBSITE',
 35: 'I-ORDINAL',
 36: 'B-LAW',
 37: 'I-LAW',
 38: 'I-WEBSITE',
 39: 'I-CURR',
 40: 'B-PRODUCT',
 41: 'I-PRODUCT'}

In [20]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
from datasets import Dataset
from datasets import load_dataset
from transformers import TrainingArguments,Trainer
from tqdm import tqdm

In [21]:
ds = Dataset.from_dict(dataset)

In [22]:
dataset = ds.train_test_split(test_size=0.2)

In [23]:
dataset_test = Dataset.from_dict(test_dataset)

In [24]:
dataset_test['tokens'][1020], dataset_test['labels'][1020]

(['جريدة', 'فلسطين', '/', 'شباط', '(', '8', '/', '2', '/', '1945', ')', '.'],
 [2, 3, 0, 4, 10, 10, 10, 10, 10, 10, 10, 0])

In [26]:
dataset['train']['tokens'][1020], dataset['train']['labels'][1020]

(['نموذج',
  'وصول',
  'الضرائب',
  'في',
  'قرية',
  'الجربا',
  'قضاء',
  'مدينة',
  'جنين',
  'لعام',
  '(',
  '1951',
  ')',
  '.'],
 [0, 0, 0, 0, 11, 25, 11, 25, 25, 4, 10, 10, 10, 0])

In [27]:
def tokenize_and_align_labels(examples, label_all_tokens=True): 
    """
    Function to tokenize and align labels with respect to the tokens. This function is specifically designed for
    Named Entity Recognition (NER) tasks where alignment of the labels is necessary after tokenization.

    Parameters:
    examples (dict): A dictionary containing the tokens and the corresponding NER tags.
                     - "tokens": list of words in a sentence.
                     - "labels": list of corresponding entity tags for each word.
                     
    label_all_tokens (bool): A flag to indicate whether all tokens should have labels. 
                             If False, only the first token of a word will have a label, 
                             the other tokens (subwords) corresponding to the same word will be assigned -100.

    Returns:
    tokenized_inputs (dict): A dictionary containing the tokenized inputs and the corresponding labels aligned with the tokens.
    """
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True) 
    labels = [] 
    for i, label in enumerate(examples["labels"]): 
        word_ids = tokenized_inputs.word_ids(batch_index=i) 
        # word_ids() => Return a list mapping the tokens
        # to their actual word in the initial sentence.
        # It Returns a list indicating the word corresponding to each token. 
        previous_word_idx = None 
        label_ids = []
        # Special tokens like `<s>` and `<\s>` are originally mapped to None 
        # We need to set the label to -100 so they are automatically ignored in the loss function.
        for word_idx in word_ids: 
            if word_idx is None: 
                # set –100 as the label for these special tokens
                label_ids.append(-100)
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            elif word_idx != previous_word_idx:
                # if current word_idx is != prev then its the most regular case
                # and add the corresponding token                 
                label_ids.append(label[word_idx]) 
            else: 
                # to take care of sub-words which have the same word_idx
                # set -100 as well for them, but only if label_all_tokens == False
                label_ids.append(label[word_idx] if label_all_tokens else -100) 
                # mask the subword representations after the first subword
                 
            previous_word_idx = word_idx 
        labels.append(label_ids) 
    tokenized_inputs["labels"] = labels 
    return tokenized_inputs 

In [28]:
q = tokenize_and_align_labels(dataset['train'][4:5]) 
q

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'input_ids': [[2, 8999, 1, 3531, 16, 6264, 3307, 12, 26, 19, 2667, 19, 2464, 8903, 13, 18, 3]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[-100, 2, 3, 3, 0, 4, 10, 10, 10, 10, 10, 10, 10, 10, 10, 0, -100]]}

In [29]:
for token, label in zip(tokenizer.convert_ids_to_tokens(q["input_ids"][0]),q["labels"][0]): 
    print(f"{label:_<40} {token}") 

-100____________________________________ [CLS]
2_______________________________________ جريدة
3_______________________________________ [UNK]
3_______________________________________ الشرق
0_______________________________________ ,
4_______________________________________ تشرين
10______________________________________ الاول
10______________________________________ (
10______________________________________ 6
10______________________________________ /
10______________________________________ 10
10______________________________________ /
10______________________________________ 19
10______________________________________ ##27
10______________________________________ )
0_______________________________________ .
-100____________________________________ [SEP]


In [14]:
tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)

AttributeError: 'dict' object has no attribute 'map'

In [32]:
tokenized_test_dataset = dataset_test.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/3304 [00:00<?, ? examples/s]

In [33]:
for token, label in zip(tokenizer.convert_ids_to_tokens(tokenized_test_dataset["input_ids"][1020]),tokenized_test_dataset["labels"][1020]): 
    print(f"{label:_<40} {token}") 

-100____________________________________ [CLS]
2_______________________________________ جريدة
3_______________________________________ فلسطين
0_______________________________________ /
4_______________________________________ شباط
10______________________________________ (
10______________________________________ 8
10______________________________________ /
10______________________________________ 2
10______________________________________ /
10______________________________________ 19
10______________________________________ ##45
10______________________________________ )
0_______________________________________ .
-100____________________________________ [SEP]


In [34]:
from transformers import TrainingArguments, Trainer 


In [35]:
data_collator = DataCollatorForTokenClassification(tokenizer) 

In [41]:
metric = datasets.load_metric("seqeval") 

/opt/conda/lib/python3.10/site-packages/datasets/load.py:756: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [42]:
example = tokenized_dataset['train'][5]

In [43]:
example

{'tokens': ['كان',
  'نصروها',
  'زمان',
  '.',
  '.',
  '.',
  'زهقنا',
  'وزهقت',
  'القدس',
  'من',
  'قلة',
  'الافعال'],
 'labels': [-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 11, 0, 0, 0, -100],
 'input_ids': [2,
  2055,
  5023,
  3736,
  6285,
  18,
  18,
  18,
  11266,
  7477,
  2300,
  1028,
  2039,
  5132,
  1908,
  15676,
  26432,
  3],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [44]:
labels_list = label_ids.values()
labels_num = len(labels_list)
print(labels_num)

42


In [45]:
model = AutoModelForTokenClassification.from_pretrained(model_path, num_labels=labels_num)

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of BertForTokenClassification were not initialized from the model checkpoint at CAMeL-Lab/bert-base-arabic-camelbert-mix and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [47]:
label_list = list(label_ids.values())
label_list

['O',
 'B-CARDINAL',
 'B-ORG',
 'I-ORG',
 'B-DATE',
 'B-LANGUAGE',
 'B-NORP',
 'B-PERS',
 'I-PERS',
 'B-OCC',
 'I-DATE',
 'B-GPE',
 'B-EVENT',
 'I-EVENT',
 'I-OCC',
 'I-CARDINAL',
 'B-FAC',
 'I-FAC',
 'B-LOC',
 'B-MONEY',
 'I-MONEY',
 'B-ORDINAL',
 'I-LANGUAGE',
 'B-TIME',
 'I-TIME',
 'I-GPE',
 'I-LOC',
 'I-NORP',
 'B-QUANTITY',
 'B-PERCENT',
 'I-PERCENT',
 'B-UNIT',
 'I-QUANTITY',
 'B-CURR',
 'B-WEBSITE',
 'I-ORDINAL',
 'B-LAW',
 'I-LAW',
 'I-WEBSITE',
 'I-CURR',
 'B-PRODUCT',
 'I-PRODUCT']

In [48]:
example["labels"]

[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 11, 0, 0, 0, -100]

In [49]:
labels = [label_list[i % len(label_list)] for i in example["labels"]]

metric.compute(predictions=[labels], references=[labels])


{'GPE': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'LOC': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [50]:
def compute_metrics(eval_preds): 
    """
    Function to compute the evaluation metrics for Named Entity Recognition (NER) tasks.
    The function computes precision, recall, F1 score and accuracy.

    Parameters:
    eval_preds (tuple): A tuple containing the predicted logits and the true labels.

    Returns:
    A dictionary containing the precision, recall, F1 score and accuracy.
    """
    pred_logits, labels = eval_preds 
    
    pred_logits = np.argmax(pred_logits, axis=2) 
    # the logits and the probabilities are in the same order,
    # so we don’t need to apply the softmax
    
    # We remove all the values where the label is -100
    predictions = [ 
        [label_list[eval_preds] for (eval_preds, l) in zip(prediction, label) if l != -100] 
        for prediction, label in zip(pred_logits, labels) 
    ] 
    
    true_labels = [ 
      [label_list[l] for (eval_preds, l) in zip(prediction, label) if l != -100] 
       for prediction, label in zip(pred_logits, labels) 
   ] 
    results = metric.compute(predictions=predictions, references=true_labels) 
    return { 
   "precision": results["overall_precision"], 
   "recall": results["overall_recall"], 
   "f1": results["overall_f1"], 
  "accuracy": results["overall_accuracy"], 
  } 

In [140]:
args = TrainingArguments( 
"test-ner",
evaluation_strategy = "epoch", 
learning_rate=2e-5, 
per_device_train_batch_size=32, 
per_device_eval_batch_size=32, 
num_train_epochs=5, 
weight_decay=0.01,
report_to='none'   
) 

In [141]:
trainer = Trainer( 
    model, 
    args, 
   train_dataset=tokenized_dataset['train'], 
   eval_dataset=tokenized_dataset['test'], 
   data_collator=data_collator, 
   tokenizer=tokenizer, 
   compute_metrics=compute_metrics,
) 

In [144]:
torch.cuda.empty_cache()

In [145]:
trainer.train() 

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.130415,0.866583,0.880896,0.873681,0.963435
2,0.094300,0.130569,0.871193,0.891367,0.881165,0.964589
3,0.094300,0.141230,0.874896,0.899029,0.886799,0.965667
4,0.053100,0.138853,0.895883,0.896646,0.896264,0.967673
5,0.053100,0.142888,0.886118,0.901584,0.893784,0.967220


/opt/conda/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but 

TrainOutput(global_step=1450, training_loss=0.06101369528934873, metrics={'train_runtime': 1134.3371, 'train_samples_per_second': 81.545, 'train_steps_per_second': 1.278, 'total_flos': 5319248661435888.0, 'train_loss': 0.06101369528934873, 'epoch': 5.0})

In [146]:
eval_results = trainer.evaluate()

In [147]:
test_results = trainer.evaluate(tokenized_test_dataset)

In [148]:
df = pd.DataFrame([test_results])
df

,eval_loss,eval_precision,eval_recall,eval_f1,eval_accuracy,eval_runtime,eval_samples_per_second,eval_steps_per_second,epoch
0,0.128084,0.872526,0.899858,0.885981,0.9666,14.3524,230.206,3.623,5.0


In [149]:
df_eval = pd.DataFrame([eval_results])
df_eval

,eval_loss,eval_precision,eval_recall,eval_f1,eval_accuracy,eval_runtime,eval_samples_per_second,eval_steps_per_second,epoch
0,0.142888,0.886118,0.901584,0.893784,0.96722,22.5986,204.658,3.23,5.0
